In [1]:
cd /home/ubuntu/DistMEC/

/home/ubuntu/DistMEC


In [2]:
import numpy as np
import matplotlib.pyplot as plt

from classes.User import *
from classes.solver import *
from classes.Server import *
from classes.distributed_utils import *

from tqdm import tqdm, tqdm_notebook
from time import sleep
import pickle
import os

In [19]:
# Fix P
def fix_P(user):
    P = user.P
    
    for i in range(user.P.shape[1]):
        tot = np.sum(user.P[i])
        if tot != 1.0:
            user.P[i] = user.P[i]/tot
        
    return


def obtain_w_stationary(Users, num_users, num_svrs):
    
    w_curr = np.zeros([num_users,num_svrs])
    for i in range(num_users):
        w_curr[i] = Users[i].stationary_reward_scale
    
    return w_curr

def extract_centralized_case(Users, num_users, num_svrs):
    
    rewards_record = np.zeros([num_users,num_svrs])
    pulls_record = np.zeros([num_users,num_svrs])
    ucb = np.zeros([num_users,num_svrs])
    
    for i in range(num_users):
        rewards_record[i] = Users[i].param_summed
        pulls_record[i] = Users[i].pulls
        ucb[i] = Users[i].ucb_raw 
    
    return rewards_record, pulls_record, ucb

In [23]:
# System Parameters
T = 5000
num_users = 10
num_svrs = 16
num_locs = 16
mu = np.random.uniform(low=0, high = 1, size = [num_users, num_svrs])
ceiling = 20
# w = np.random.uniform(low=0, high=1, size= [num_users, num_svrs])
space_1d_dim = 10 #4 for small, 10 for big

num_explore = 1
trials = 1

# result dictionaries
result_dicts = []
performance_list = []
user_dicts = []

save_name = 'results/22_06_16_ceiling_20/'
isExist = os.path.exists(save_name)
if not isExist:
  # Create a new directory because it does not exist 
    os.makedirs(save_name)

for tri in range(trials):
    
    print("trial:", tri)
    result_dict = {}
    user_dict = {}
    
    # Generate visitng locations and server locations
    usr_locs = gen_eq_locs(space_1d=space_1d_dim, nums=num_locs)
    svr_locs = gen_eq_locs(space_1d=space_1d_dim, nums=num_svrs)

    # Create Users
    Users_w = []
    for i in range(num_users):
        Users_w += [User(usr_locs,svr_locs,mu[i],i,
                      max_dist = 4, threshold_dist = 4, self_weight = 1 - 0.95, P = None, ceiling = 1,
                      sticky_mode = False, kick_mode=True)]
        
    # Create Servers
    Servers = []
    for i in range(num_svrs):
        Servers += [Server(svr_locs[i],mu,i)]

    # Recorded Values - reservation
    regret_w = np.zeros(T)
    collision_count_w = np.zeros(T)
    optimal_reward_w = np.zeros(T)

    # Explore rounds are common/shared across all users
    explore_rounds(Users_w, num_users, Servers, mu, regret_w, collision_count_w, optimal_reward_w,
                   usr_move_flag = True, rounds=num_explore)
    
    # Distributed - true w learning
    Users_s = copy.deepcopy(Users_w)
    regret_s = copy.deepcopy(regret_w)
    collision_count_s = copy.deepcopy(collision_count_w)
    for u in Users_s:
        u.sticky_mode = True
        u.kick_mode = True #false
        
    # Centralized - true w learning
    rewards_record_ct, pulls_record_ct, ucb_ct = extract_centralized_case(Users_w, num_users, num_svrs)
    regret_ct = copy.deepcopy(regret_w)
    
    # Centralized - stationary w learning
    rewards_record_cs, pulls_record_cs, ucb_cs = extract_centralized_case(Users_w, num_users, num_svrs)
    regret_cs = copy.deepcopy(regret_w)
    
    # Centralized - stationary w, known mu
    w_stat = obtain_w_stationary(Users_w, num_users, num_svrs)
    optimal_stat = 

trial: 0


In [21]:
fix_P(Users_w[0])

In [26]:
w_stat

array([[0.57339818, 0.66409911, 0.70770368, 0.66932211, 0.64123982,
        0.74729767, 0.80304323, 0.74186793, 0.6539174 , 0.75118508,
        0.79705425, 0.7559708 , 0.61340296, 0.69595359, 0.71052397,
        0.66808567],
       [0.69179988, 0.75706362, 0.7361816 , 0.63690942, 0.73846895,
        0.79354971, 0.75976818, 0.66767132, 0.71705474, 0.76762157,
        0.72596659, 0.63849289, 0.61697845, 0.66020866, 0.63816013,
        0.55944072],
       [0.62807575, 0.71659733, 0.7596903 , 0.7021259 , 0.67418199,
        0.77484904, 0.83580099, 0.77114445, 0.6507128 , 0.74041612,
        0.77761761, 0.73181914, 0.57026209, 0.65154826, 0.67364741,
        0.63020747],
       [0.59166675, 0.6710131 , 0.71930391, 0.69309561, 0.62629433,
        0.71615032, 0.75091639, 0.71737575, 0.62016352, 0.70910304,
        0.73480444, 0.6932407 , 0.56813405, 0.66409188, 0.66185419,
        0.61501224],
       [0.62696219, 0.68545871, 0.7039331 , 0.67702095, 0.66692377,
        0.73966617, 0.74334221, 

In [160]:
P = np.array([[0.7, 0.2, 0.1],[0.4,0.6,0],[0,1,0]])

In [161]:
evals, evecs = np.linalg.eig(Users[0].P.T)
evec1 = evecs[:,np.isclose(evals, 1)]

#Since np.isclose will return an array, we've indexed with an array
#so we still have our 2nd axis.  Get rid of it, since it's only size 1.
evec1 = evec1[:,0]

stationary = evec1 / evec1.sum()

#eigs finds complex eigenvalues and eigenvectors, so you'll want the real part.
stationary = stationary.real

In [162]:
s2 = stationary.reshape([1,stationary.shape[0]])

In [163]:
r2 = Users[0].reward_scale

In [174]:
true_scaling = np.matmul(s2, Users[0].reward_scale)

In [178]:
stationary

array([0.01418415, 0.04076948, 0.07075752, 0.11449394, 0.01854684,
       0.17635562, 0.00885521, 0.01378636, 0.17506885, 0.07949673,
       0.03025502, 0.01886457, 0.01227082, 0.16134263, 0.01981257,
       0.0451397 ])

In [12]:
w2 = obtain_w_stationary(Users_w,  num_users, num_svrs)

In [13]:
w2

array([[0.68966886, 0.70309462, 0.66513838, 0.58121115, 0.73974057,
        0.77837824, 0.72120058, 0.62917617, 0.72003593, 0.75355825,
        0.7159357 , 0.63020159, 0.64226993, 0.68186278, 0.65710858,
        0.57300329],
       [0.61681633, 0.68513614, 0.69012004, 0.63406074, 0.68178213,
        0.74668027, 0.74460606, 0.69157697, 0.70395873, 0.73939451,
        0.73369552, 0.6761866 , 0.61230602, 0.6642079 , 0.66341463,
        0.61814077],
       [0.57644049, 0.6620544 , 0.67724389, 0.6350632 , 0.65112569,
        0.75178164, 0.77793392, 0.72907838, 0.67127607, 0.77159917,
        0.80676505, 0.76935531, 0.62287109, 0.71523508, 0.7371109 ,
        0.68963639],
       [0.59506308, 0.6564949 , 0.67512521, 0.625099  , 0.66596378,
        0.73424469, 0.73989736, 0.69225131, 0.6884668 , 0.74150859,
        0.75422249, 0.69209454, 0.62961885, 0.68181368, 0.68199131,
        0.63592823],
       [0.662027  , 0.68969299, 0.66908196, 0.58861805, 0.70488892,
        0.76752797, 0.73191875, 